In [35]:
%matplotlib inline
import numpy as np
import sys
sys.path.append('./../../flux/')
from coare3 import coare3
from coare4 import coare4
from gravity_constant import grv
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seawater as sw
import cartopy.crs as ccrs                   # import projections
import cartopy.feature as cf                 # import features

fig_dir='C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/Saildrone/'

icefile='C:/Users/gentemann/Google Drive/f_drive/docs/projects/misst-arctic/Ice Present.xlsx'

data_dir = 'F:/data/cruise_data/saildrone/2019_arctic/post_mission/'

adir_sbe='F:/data/cruise_data/saildrone/2019_arctic/sbe56/sd-'

data_dir_sbe_combined = 'F:/data/cruise_data/saildrone/2019_arctic/post_mission_combined/'

#    Args:  
#        u (float): relative wind speed (m/s) at height zu(m)
#        t (float): bulk air temperature (degC) at height zt(m)
#        rh (float): relative humidity (#) at height zq(m) [0-100]
#        P (float): surface air pressure (mb) (default = 1015)
#        ts (float): water temperature (degC) see jcool below
#        Rs (float): downward shortwave radiation (W/m^2) (default = 150) 
#        Rl (float): downward longwave radiation (W/m^2) (default = 370)
#        lat (float): latitude (default = +45 N)
#        zi (float): PBL height (m) (default = 600m)

jcool = 1 #ocean bulk temperature


inputs = {'u':np.array([12.5]),'ts':np.array([28.48]),'Q':np.array([4.56])}
res = coare4(inputs)


In [7]:
ds = xr.open_mfdataset(data_dir_sbe_combined+'*.nc',combine='nested',concat_dim='trajectory').load()
ds

<xarray.Dataset>
Dimensions:                         (time: 215731, trajectory: 2)
Coordinates:
  * time                            (time) datetime64[ns] 2019-05-14T23:00:00 ... 2019-10-11T18:30:00
    lat                             (trajectory, time) float64 53.85 ... 53.84
    lon                             (trajectory, time) float64 -166.6 ... -166.6
  * trajectory                      (trajectory) object '1036' '1037'
Data variables:
    SOG                             (trajectory, time) float64 1.41 ... 0.005
    SOG_FILTERED_MEAN               (trajectory, time) float64 1.194 ... 0.028
    SOG_FILTERED_STDDEV             (trajectory, time) float64 0.096 ... 0.02
    SOG_FILTERED_MAX                (trajectory, time) float64 1.48 ... 0.095
    SOG_FILTERED_MIN                (trajectory, time) float64 1.041 ... 0.002
    COG                             (trajectory, time) float64 324.0 ... 93.4
    COG_FILTERED_MEAN               (trajectory, time) float64 324.6 ... 83.2
    COG_

In [41]:
#    Args:  
#        u (float): relative wind speed (m/s) at height zu(m)
#        t (float): bulk air temperature (degC) at height zt(m)
#        rh (float): relative humidity (#) at height zq(m) [0-100]
#        P (float): surface air pressure (mb) (default = 1015)
#        ts (float): water temperature (degC) see jcool below
#        Rs (float): downward shortwave radiation (W/m^2) (default = 150) 
#        Rl (float): downward longwave radiation (W/m^2) (default = 370)
#        lat (float): latitude (default = +45 N)
#        zi (float): PBL height (m) (default = 600m)

jcool = 1 #ocean bulk temperature


for i in range(1):
    tem = ds.isel(trajectory=i)
    
    cond = (np.isfinite(tem.wspd_MEAN) & np.isfinite(tem.TEMP_AIR_MEAN) \
            & np.isfinite(tem.RH_MEAN) & np.isfinite(tem.BARO_PRES_MEAN) \
            & np.isfinite(tem.sea_water_temperature_00_mean) \
            & np.isfinite(tem.lat))                                                                            
    ds2=tem.where(cond,drop=True)
    print(ds2.dims)
    inputs = {'u':ds2.wspd_MEAN,'t':ds2.TEMP_AIR_MEAN,'rh':ds2.RH_MEAN,
              'P':ds2.BARO_PRES_MEAN,
              'ts':ds2.sea_water_temperature_00_mean,'lat':ds2.lat}
    res = coare4(inputs)


Frozen(SortedKeysDict({'time': 125718}))


MemoryError: Unable to allocate array with shape (125718, 125718) and data type float64

In [37]:
grav = grv(ds2.lat,shape_wanted=ds2.lat.shape)

In [39]:
grav = grv(ds2.lat,shape_wanted=ds2.sea_water_temperature_00_mean.shape)
